# Code to compare different M1M3 settings.  See SITCOM-1160.

Craig Lage 26-Dec-23

In [ ]:
import sys, time, os, asyncio, glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import openpyxl as xl
from astropy.time import Time, TimeDelta
from lsst_efd_client import EfdClient
from lsst.ts.xml.tables.m1m3 import FATable, FAIndex, force_actuator_from_id, actuator_id_to_index
from lsst.summit.utils.efdUtils import calcNextDay
from lsst.summit.utils import getCurrentDayObs_int, dayObsIntToString
from lsst.summit.utils.tmaUtils import TMAEventMaker, getAzimuthElevationDataForEvent
from lsst.summit.utils.blockUtils import BlockParser


In [ ]:
#This sets the column headers in the spreadsheet
groups = [["Hardpoints",["HP Name", "HP Min", "HP Max"]],
          ["MTMount",["MT Name", "MT Min", "MT Max"]],
          ["Actuator Following Error Summary",["AS ID", "AS P Min", "AS P Max", \
                                              "AS S Min", "AS S Max"]],
          ["Actuator Following Errors",["A ID", "A P Min", "A P Max", \
                                       "A S Min", "A S Max"]]]

addBlankColumn = True # Adds a blank column after each group for readability
firstRow = 2 # Where to place the dataframe in the spreadsheet
firstColumn = 0 # Where to place the dataframe in the spreadsheet

filename = "/home/c/cslage/u/MTM1M3/data/MTM1M3_Block_178_04Jan24.xlsx"

# Cells below are the functions that do the work

In [ ]:
def formatSpreadsheet(filename, sheetName, columns):
    workbook = xl.load_workbook(filename=filename)
    sheet = workbook[sheetName]
    start_column = 1
    for [name, headings] in groups:
        end_column = start_column + len(headings) - 1
        sheet.merge_cells(start_row=2, end_row=2, 
                          start_column=start_column, end_column=end_column)
        cell = sheet.cell(row=2, column=start_column)  
        cell.value = name  
        cell.alignment = xl.styles.Alignment(horizontal='center', vertical='center', wrapText=True)  
        fontStyle = xl.styles.Font(bold=True, size = "10")
        cell.font = fontStyle
        cell.border = xl.styles.Border(outline=True) 
        start_column = end_column + 2
    for heading_column in range(1, end_column + 1):
        cell = sheet.cell(row=3, column=heading_column)
        fontStyle = xl.styles.Font(bold=True, size = "9")
        cell.font = fontStyle
        
    sheet.merge_cells(start_row=1, end_row=1, 
                      start_column=1, end_column=end_column)
    cell = sheet.cell(row=1, column=1)  
    cell.value = sheetName  
    cell.alignment = xl.styles.Alignment(horizontal='center', vertical='center')  
    fontStyle = xl.styles.Font(bold=True, size = "18")
    cell.font = fontStyle
    sheet.row_dimensions[1].height = 30
    sheet.row_dimensions[2].height = 20
    #sheet.print_area = 'A1:O20'
    for i, column in enumerate(columns):
        columnLetter = xl.utils.get_column_letter(i+1)
        col = sheet.column_dimensions[columnLetter]
        if column == '':
            col.width = 2
        elif 'HP Name' in column:
            col.width = 15
        elif 'HP Min' in column or 'HP Max' in column:
            col.width = 10
            col.number_format = '0.0'
        elif 'ID' in column:
            col.width = 8
            col.number_format = '0'
        else:
            col.width = 8
            #print("Got here, column = ", column, columnLetter)
            col.number_format = '0.0'

    workbook.save(filename=filename)
    return workbook

In [ ]:
def addColumn(df, columnName, columnData):
    # Allows to add a new column longer or shorter 
    # than the current dataframe
    newLength = len(columnData)
    currentLength = len(df.index)
    #print(columnName, currentLength, newLength)
    if newLength < currentLength:
        columnData.extend(['']*(currentLength - newLength))
    elif currentLength < newLength:
        df = df.reindex(pd.RangeIndex(start=0, stop=newLength), fill_value='')
    df[columnName] = columnData
    return df


In [ ]:
def fivePointStencil(pos, times):
    # Numerically differentiates a data stream
    der = np.zeros(len(pos))
    for i in range(2, len(pos)-2):
        der[i] = pos[i-2] - 8.0*pos[i-1] + 8.0*pos[i+1] - pos[i+2]
        der[i] / 12.0*(times[i] - times[i-1])
    return der


In [ ]:
async def getHardPointData(df, client, events):
    for i, event in enumerate(events):
        start = event.begin
        end = event.end
        this_ret = await client.select_time_series(
            "lsst.sal.MTM1M3.hardpointActuatorData",
            ["timestamp"]
            + [f"measuredForce{hp}" for hp in range(6)]
            + [f"f{a}" for a in "xyz"]
            + [f"m{a}" for a in "xyz"],
            start,
            end,
            )
        if i == 0:
            ret = this_ret
        else:
            ret = pd.concat([ret, this_ret])

    names = []
    mins = []
    maxs = []
    for hp in range(6):
        name = f"measuredForce{hp}"
        data = ret[name]
        min = np.min(data)
        max = np.max(data)
        names.append(name)
        mins.append(min)
        maxs.append(max)
    for a in "xyz":
        name = f"f{a}"
        data = ret[name]
        min = np.min(data)
        max = np.max(data)        
        names.append(name)
        mins.append(min)
        maxs.append(max)
    for a in "xyz":
        name = f"m{a}"
        data = ret[name]
        min = np.min(data)
        max = np.max(data)        
        names.append(name)
        mins.append(min)
        maxs.append(max)

    df = addColumn(df, "HP Name", names)
    df = addColumn(df, "HP Min", mins)
    df = addColumn(df, "HP Max", maxs)
    return df


In [ ]:
async def getFollowingErrors(df, client, events):
    ids = []
    primaryMins = []
    primaryMaxs = []
    secondaryMins = []
    secondaryMaxs = []
    for i, event in enumerate(events):
        start = event.begin
        end = event.end

        these_forces = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", \
                                         ['*'], start, end)
        if i == 0:
            forces = these_forces
        else:
            forces = pd.concat([forces, these_forces])

    for index in range(len(FATable)):
        try:
            id = FATable[index].actuator_id
            fa = force_actuator_from_id(id)
            primary_follow = f"primaryCylinderFollowingError{fa.index}"
            if fa.actuator_type.name == 'DAA':
                secondary_follow = f"secondaryCylinderFollowingError{fa.s_index}"
                secondary_name = fa.orientation.name
            else:
                secondary_follow = None
            primaryMin = np.min(forces[primary_follow].values)
            primaryMins.append(primaryMin)
            primaryMax = np.max(forces[primary_follow].values)
            primaryMaxs.append(primaryMax)
            ids.append(id)
            if secondary_follow:
                secondaryMin = np.min(forces[secondary_follow].values)
                secondaryMins.append(secondaryMin)
                secondaryMax = np.max(forces[secondary_follow].values)
                secondaryMaxs.append(secondaryMax)
            else:
                secondaryMins.append(np.nan)
                secondaryMaxs.append(np.nan)
        except:
            continue
    primaryArgMin = np.argmin(np.array(primaryMins))
    primaryArgMax = np.argmax(np.array(primaryMaxs))
    secondaryArgMin = np.nanargmin(np.array(secondaryMins))
    secondaryArgMax = np.nanargmax(np.array(secondaryMaxs))
    ['' if x is np.nan else x for x in secondaryMins]
    ['' if x is np.nan else x for x in secondaryMaxs]
    df = addColumn(df, "A ID", ids)
    df = addColumn(df, "A P Min", primaryMins)
    df = addColumn(df, "A P Max", primaryMaxs)
    df = addColumn(df, "A S Min", secondaryMins)
    df = addColumn(df, "A S Max", secondaryMaxs)
    #print(secondaryMins)
    #print(secondaryArgMax, secondaryArgMin)
    # Now make the global Min/Max summary
    ids = [ids[primaryArgMin], ids[primaryArgMax],ids[secondaryArgMin],ids[secondaryArgMax]]
    primaryMins = [primaryMins[primaryArgMin],'','','']
    primaryMaxs = ['', primaryMaxs[primaryArgMax],'','']
    secondaryMins = ['','', secondaryMins[secondaryArgMin],'']
    secondaryMaxs = ['','','', secondaryMaxs[secondaryArgMax]]
    df = addColumn(df, "AS ID", ids)
    df = addColumn(df, "AS P Min", primaryMins)
    df = addColumn(df, "AS P Max", primaryMaxs)
    df = addColumn(df, "AS S Min", secondaryMins)
    df = addColumn(df, "AS S Max", secondaryMaxs)
    
    return df


In [ ]:
async def getTMAData(df, client, events):
    minAzV=0.0; minAzA=0.0; minElV=0.0; minElA=0.0; minAngX=0.0; minAngY=0.0; minAngZ=0.0;
    maxAzV=0.0; maxAzA=0.0; maxElV=0.0; maxElA=0.0; maxAngX=0.0; maxAngY=0.0; maxAngZ=0.0;

    for i, event in enumerate(events):
        start = event.begin
        end = event.end
        az = await client.select_time_series('lsst.sal.MTMount.azimuth', \
                                                    ['*'],  start, end)
        el = await client.select_time_series('lsst.sal.MTMount.elevation', \
                                                    ['*'],  start, end) 
        #print(event.seqNum, start, end, len(az), len(el))
    
        acc = await client.select_time_series('lsst.sal.MTM1M3.accelerometerData', \
                                                    ['*'],  start, end)     
        # NGet the velocity and differentiate it to get the acceleration
        azVs = az['actualVelocity'].values
        azXs = az['timestamp'].values - az['timestamp'].values[0]
        azAs = fivePointStencil(azVs, azXs)
        elVs = el['actualVelocity'].values
        elXs = el['timestamp'].values - el['timestamp'].values[0]
        elAs = fivePointStencil(elVs, elXs)
        accAx = acc['angularAccelerationX'].values
        accAy = acc['angularAccelerationY'].values
        accAz = acc['angularAccelerationZ'].values    
        minAzV = min(minAzV, np.min(azVs))
        maxAzV = max(maxAzV, np.max(azVs))
        minAzA = min(minAzA, np.min(azAs))
        maxAzA = max(maxAzA, np.max(azAs))
        minElV = min(minElV, np.min(elVs))
        maxElV = max(maxElV, np.max(elVs))
        minElA = min(minElA, np.min(elAs))
        maxElA = max(maxElA, np.max(elAs))
        minAngX = min(minAngX, np.min(accAx))
        maxAngX = max(maxAngX, np.max(accAx))
        minAngY = min(minAngY, np.min(accAy))
        maxAngY = max(maxAngY, np.max(accAy))
        minAngZ = min(minAngZ, np.min(accAz))
        maxAngZ = max(maxAngZ, np.max(accAz))
    names = ['Az Vel', 'Az Acc', 'El Vel', 'El Acc', 'AngAccX', 'AngAccY', 'AngAccZ']
    mins = [minAzV, minAzA, minElV, minElA, minAngX, minAngY, minAngZ]
    maxs = [maxAzV, maxAzA, maxElV, maxElA, maxAngX, maxAngY, maxAngZ]
    df = addColumn(df, "MT Name", names)
    df = addColumn(df, "MT Min", mins)
    df = addColumn(df, "MT Max", maxs)
    return df
          

# The cell below creates and formats the spreadsheet

In [ ]:
client = EfdClient("usdf_efd")
eventMaker = TMAEventMaker()
dayObs = 20240104
blockNum = 322        
events = eventMaker.getEvents(dayObs)
#events = [e for e in events if e.relatesTo(block=blockNum)] 

jerks = [[20,10],[10,5],[5,2.5],[1.0,0.5]]
settings = ["GGRG", "GRRG", "GGRR", "GRRR"]
startSeqNum = [196,204,212,227,240,248,256,264,274,289,297,312,322,336,344,379]

dfDict = {}
counter = 0
for [azJerk,elJerk] in jerks:
    for setting in settings:
        key = f"AzJ-{azJerk:.1f},ElJ-{elJerk:.1f},{setting}"
        print(key)
        seqNumList = list(range(startSeqNum[counter], startSeqNum[counter] + 7))
        theseEvents = [e for e in events if e.seqNum in seqNumList]
        print(key, [e.seqNum for e in theseEvents])

        sheetName = key
        columns = []
        for [name, headings] in groups:
            for heading in headings:
                columns.append(heading)
            if addBlankColumn:
                columns.append('')
    
        df = pd.DataFrame(columns=columns)
        df = await getHardPointData(df, client, theseEvents)
        df = await getTMAData(df, client,theseEvents)
        df = await getFollowingErrors(df, client, theseEvents)
        if counter == 0:
            with pd.ExcelWriter(filename) as writer:  
                df.to_excel(writer, sheet_name=sheetName, startrow=firstRow, \
                    startcol=firstColumn, index=False)
        else:
            with pd.ExcelWriter(filename, mode='a') as writer:  
                df.to_excel(writer, sheet_name=sheetName, startrow=firstRow, \
                    startcol=firstColumn, index=False)
    
        workbook = formatSpreadsheet(filename, sheetName, columns)
        
        dfDict[key] = df        
        counter += 1


In [ ]:
dfDict

In [ ]:
%matplotlib inline
fig = plt.figure(figsize=(10,5))
ax = fig.add_axes(
    (0.1, 0.45, 0.8, 0.45))

    
ax.set_title("Block 179 testing - 20240104", fontsize = 24)
ax.scatter(xaxis, HPs)
ax.set_xticks(ticks=list(range(16)),labels=keys, rotation=90, fontsize=12)
ax.set_ylabel("HP max forces (N)")
                       
plt.savefig('/home/c/cslage/u/MTM1M3/data/Block_179_HP_04Jan24.png')